In [11]:
# https://www.tensorflow.org/decision_forests/tutorials/beginner_colab
# https://www.tensorflow.org/decision_forests/tutorials/predict_colab
# https://www.tensorflow.org/decision_forests/tutorials/automatic_tuning_colab
# https://www.tensorflow.org/decision_forests/tutorials/dtreeviz_colab
from google.cloud import aiplatform
from google.cloud import bigquery

import pandas as pd
import numpy as np

import math
import os

import tensorflow as tf
print(tf.__version__)

2.12.0


In [12]:
# ! pip install --upgrade tensorflow-decision-forests
import tensorflow_decision_forests as tfdf
print(tfdf.__version__)

1.3.0


# Variable to Configuration

In [13]:
projectId='pongthorn'
dataset_id='SMartML'

train_name='train2_incident'
test_name='test2_incident'

train_table_id=f"{projectId}.{dataset_id}.{train_name}"
test_tabel_id=f"{projectId}.{dataset_id}.{test_name}"
print(f"train-ds = {train_table_id}")
print(f"test-ds = {test_tabel_id}")

train-ds = pongthorn.SMartML.train2_incident
test-ds = pongthorn.SMartML.test2_incident


In [14]:
metric="accuracy"
model_gs_path="gs://demo-tuned-tf-incident-pongthorn/model_df_tf"

In [15]:
unusedCols=['severity_id','severity_name','label_binary_severity','open_to_close_hour','response_to_resolved_hour']
cateCols=['sla','product_type','brand','service_type','incident_type','range_open_to_close_hour','range_response_to_resolved_hour']
numbericCols=[]

# unusedCols=['severity_id','severity_name','label_binary_severity','range_open_to_close_hour','range_response_to_resolved_hour']
# cateCols=['sla','product_type','brand','service_type','incident_type']
# numbericCols=['open_to_close_hour','response_to_resolved_hour']


labelCol='label_multi_severity'
# labelCol='label_binary_severity'

# Load & Prepare Data

In [16]:
def load_ml_data(data_path):
 df=pd.read_csv(data_path)
 df =df.drop(columns=unusedCols)
 
 return df

def load_data_bq(sql:str):
 
 query_result=client.query(sql)
 df=query_result.to_dataframe()
 df =df.drop(columns=unusedCols)
 df[labelCol]=df[labelCol].astype('int') 
 df=df[[labelCol]+cateCols+numbericCols]   
  
 return df

In [17]:
client = bigquery.Client(project=projectId)

train=load_data_bq(f"SELECT * FROM {train_table_id}")
test=load_data_bq(f"SELECT * FROM {test_tabel_id}")

print(train.info())

print(test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2076 entries, 0 to 2075
Data columns (total 8 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   label_multi_severity             2076 non-null   int64 
 1   sla                              2076 non-null   object
 2   product_type                     2076 non-null   object
 3   brand                            2076 non-null   object
 4   service_type                     2076 non-null   object
 5   incident_type                    2076 non-null   object
 6   range_open_to_close_hour         2076 non-null   object
 7   range_response_to_resolved_hour  2076 non-null   object
dtypes: int64(1), object(7)
memory usage: 129.9+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 519 entries, 0 to 518
Data columns (total 8 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 

In [18]:
train.tail(5)

,label_multi_severity,sla,product_type,brand,service_type,incident_type,range_open_to_close_hour,range_response_to_resolved_hour
2071,1,24x7 4Hrs Response Time,Software,Trend Micro,Request,Software,best,best
2072,2,24x7 6Hrs Resolution Time,Software,Trend Micro,Incident,Configuration Change,good,good
2073,2,24x7 4Hrs Response Time,Software,Trend Micro,Incident,General Incident,worst,worst
2074,2,24x7 6Hrs Resolution Time,Software,Trend Micro,Incident,General Incident,moderate,moderate
2075,2,24x7 6Hrs Resolution Time,Software,Trend Micro,Incident,General Incident,best,best


In [19]:
no_unseen=9
unseen=test.iloc[-no_unseen:,:]
print(unseen.info())
unseen

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 510 to 518
Data columns (total 8 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   label_multi_severity             9 non-null      int64 
 1   sla                              9 non-null      object
 2   product_type                     9 non-null      object
 3   brand                            9 non-null      object
 4   service_type                     9 non-null      object
 5   incident_type                    9 non-null      object
 6   range_open_to_close_hour         9 non-null      object
 7   range_response_to_resolved_hour  9 non-null      object
dtypes: int64(1), object(7)
memory usage: 708.0+ bytes
None


,label_multi_severity,sla,product_type,brand,service_type,incident_type,range_open_to_close_hour,range_response_to_resolved_hour
510,1,24x7 6Hrs Resolution Time,Software,CIMCO-CMMS,Incident,Software,moderate,moderate
511,2,24x7 6Hrs Resolution Time,Software,CIMCO-CMMS,Incident,Software,good,good
512,2,24x7 6Hrs Resolution Time,Software,CIMCO-CMMS,Incident,General Incident,moderate,moderate
513,0,24x7 4Hrs Response Time,Software,Trend Micro,Request,General Incident,good,good
514,0,24x7 4Hrs Response Time,Software,Trend Micro,Request,General Incident,moderate,good
515,0,24x7 4Hrs Response Time,Software,Trend Micro,Request,General Incident,best,best
516,1,24x7 4Hrs Response Time,Software,Trend Micro,Incident,Software,best,best
517,1,24x7 4Hrs Response Time,Software,Trend Micro,Incident,Software,good,good
518,1,24x7 4Hrs Response Time,Software,Trend Micro,Incident,General Incident,good,good


In [20]:
test=test.iloc[0:len(test)-no_unseen,:]
print(test.info())
test.tail()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 510 entries, 0 to 509
Data columns (total 8 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   label_multi_severity             510 non-null    int64 
 1   sla                              510 non-null    object
 2   product_type                     510 non-null    object
 3   brand                            510 non-null    object
 4   service_type                     510 non-null    object
 5   incident_type                    510 non-null    object
 6   range_open_to_close_hour         510 non-null    object
 7   range_response_to_resolved_hour  510 non-null    object
dtypes: int64(1), object(7)
memory usage: 32.0+ KB
None


,label_multi_severity,sla,product_type,brand,service_type,incident_type,range_open_to_close_hour,range_response_to_resolved_hour
505,2,24x7 4Hrs Resolution Time,Firewall,Palo Alto,Incident,Software,best,best
506,2,24x7 4Hrs Resolution Time,Firewall,Palo Alto,Incident,General Incident,best,best
507,2,24x7 4Hrs Resolution Time,Firewall,Palo Alto,Incident,General Incident,best,best
508,2,24x7 4Hrs Resolution Time,Firewall,Palo Alto,Incident,General Incident,best,best
509,2,24x7 4Hrs Response Time,Firewall,Palo Alto,Incident,General Incident,bad,bad


# Build Model

In [21]:
train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(train, label=labelCol)
test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(test, label=labelCol)

In [22]:
# Specify the model.
#model = tfdf.keras.RandomForestModel()
model=tfdf.keras.GradientBoostedTreesModel()
model.fit(x=train_ds)

Use /var/tmp/tmpd8e89vm0 as temporary training directory
Reading training dataset...


[WARNING 23-06-27 13:28:36.7303 UTC gradient_boosted_trees.cc:1797] "goss_alpha" set but "sampling_method" not equal to "GOSS".
[WARNING 23-06-27 13:28:36.7316 UTC gradient_boosted_trees.cc:1808] "goss_beta" set but "sampling_method" not equal to "GOSS".
[WARNING 23-06-27 13:28:36.7318 UTC gradient_boosted_trees.cc:1822] "selective_gradient_boosting_ratio" set but "sampling_method" not equal to "SELGB".
2023-06-27 13:28:36.812804: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_7' with dtype int64 and shape [2076]
	 [[{{node Placeholder/_7}}]]


Training dataset read in 0:00:04.593301. Found 2076 examples.
Training model...
Model trained in 0:00:00.360890
Compiling model...


[INFO 23-06-27 13:28:41.6720 UTC kernel.cc:1242] Loading model from path /var/tmp/tmpd8e89vm0/model/ with prefix cf07123b368347ab
[INFO 23-06-27 13:28:41.7284 UTC decision_forest.cc:660] Model loaded with 108 root(s), 5198 node(s), and 7 input feature(s).
[INFO 23-06-27 13:28:41.7293 UTC abstract_model.cc:1311] Engine "GradientBoostedTreesGeneric" built
[INFO 23-06-27 13:28:41.7298 UTC kernel.cc:1074] Use fast generic engine
2023-06-27 13:28:41.788070: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_6' with dtype string and shape [2076]
	 [[{{node Placeholder/_6}}]]


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Model compiled.


In [25]:
model.compile(metrics=[metric])
evaluation = model.evaluate(test_ds, return_dict=True)
for name, value in evaluation.items():
  print(f"{name}: {value:.4f}")

1/1 [==============================] - 0s 170ms/step - loss: 0.0000e+00 - accuracy: 0.7000
loss: 0.0000
accuracy: 0.7000


# Save Model

In [26]:
model.save(model_gs_path)

INFO:tensorflow:Assets written to: gs://demo-tuned-tf-incident-pongthorn/model_df_tf/assets


INFO:tensorflow:Assets written to: gs://demo-tuned-tf-incident-pongthorn/model_df_tf/assets


# Load Model

In [28]:
abc_model = tf.keras.models.load_model(model_gs_path)  

[INFO 23-06-27 13:31:51.8726 UTC kernel.cc:1242] Loading model from path gs://demo-tuned-tf-incident-pongthorn/model_df_tf/assets/ with prefix cf07123b368347ab
[INFO 23-06-27 13:31:52.3339 UTC decision_forest.cc:660] Model loaded with 108 root(s), 5198 node(s), and 7 input feature(s).
[INFO 23-06-27 13:31:52.3348 UTC abstract_model.cc:1311] Engine "GradientBoostedTreesGeneric" built
[INFO 23-06-27 13:31:52.3354 UTC kernel.cc:1074] Use fast generic engine


# Make Prediction

In [29]:
unseen_ds= tfdf.keras.pd_dataframe_to_tf_dataset(unseen.drop(columns=[labelCol]))
unseen_ds

<_PrefetchDataset element_spec={'sla': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'product_type': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'brand': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'service_type': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'incident_type': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'range_open_to_close_hour': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'range_response_to_resolved_hour': TensorSpec(shape=(None,), dtype=tf.string, name=None)}>

In [30]:
predResultList=abc_model.predict(unseen_ds)

1/1 [==============================] - 0s 108ms/step


2023-06-27 13:31:59.643796: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_6' with dtype string and shape [9]
	 [[{{node Placeholder/_6}}]]


In [31]:
map_severity_to_class={0:4,1: 3, 2: 2, 3: 1}
for predResult in predResultList:
    _class=tf.argmax(predResult,-1).numpy()
    print(f"{predResult} : {_class} as severity#{map_severity_to_class[_class]}")   

[0.01725619 0.244531   0.7207276  0.01748515] : 2 as severity#2
[0.01813892 0.24556433 0.7179297  0.01836703] : 2 as severity#2
[0.02054393 0.30821964 0.65076953 0.02046687] : 2 as severity#2
[0.8526699  0.11844265 0.01640307 0.01248433] : 0 as severity#4
[0.8532449  0.11765686 0.0161251  0.01297304] : 0 as severity#4
[0.9215788  0.05408689 0.01521658 0.00911774] : 0 as severity#4
[0.02320344 0.76125306 0.19302855 0.02251487] : 1 as severity#3
[0.03241264 0.6210357  0.31505343 0.03149821] : 1 as severity#3
[0.02731145 0.7103505  0.23605128 0.02628671] : 1 as severity#3
